# Run PESTPP-OPT

In this notebook we will setup and solve a mgmt optimization problem around how much groundwater can be pumped while maintaining sw-gw exchange

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['font.size']=12
import flopy
import pyemu


In [ ]:
t_d = "template"
m_d = "master_opt"

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
pst.write_par_summary_table(filename="none").sort_index()

define our decision varible group and also set some ++args

In [ ]:
pst.pestpp_options = {}
#dvg = ["welflux_k02","welflux"]
dvg = ["welflux_k02"]
pst.pestpp_options["opt_dec_var_groups"] = dvg
pst.pestpp_options["opt_direction"] = "max"

For the first run, we wont use chance constraints, so just fix all non-decision-variable parameter.  We also need to set some realistic bounds on the `welflux` multiplier decision variables.  Finally, we need to specify a larger derivative increment for the decision varible group

In [ ]:
par = pst.parameter_data
par.loc[:,"partrans"] = "fixed"

#turn off pumping in the scenario
par.loc["welflux_001","parlbnd"] = 0.0 
par.loc["welflux_001","parval1"] = 0.0 
dvg_pars = par.loc[par.pargp.apply(lambda x: x in dvg),"parnme"]
par.loc[dvg_pars,"partrans"] = "none"
par.loc[dvg_pars,"parlbnd"] = 0.0
par.loc[dvg_pars,"parubnd"] = 2.0
par.loc[dvg_pars,"parval1"] = 1.0

pst.rectify_pgroups()
pst.parameter_groups.loc[dvg,"inctyp"] = "absolute"
pst.parameter_groups.loc[dvg,"inctyp"] = "absolute"
pst.parameter_groups.loc[dvg,"derinc"] = 0.25

pst.parameter_groups.loc[dvg,:]

### define constraints

model-based constraints are identified in pestpp-opt by an obs group that starts with "less_than" or "greater_than" and a weight greater than zero.  So first, we turn off all of the weights and get names for the sw-gw exchange observations

In [ ]:
obs = pst.observation_data
obs.loc[:,"weight"] = 0.0
swgw_hist = obs.loc[obs.obsnme.apply(lambda x: "fa" in x and( "hw" in x or "tw" in x)),"obsnme"]
obs.loc[swgw_hist,:]

We need to change the obs group (`obgnme`) so that `pestpp-opt` will recognize these two model outputs as constraints.  The `obsval` becomes the RHS of the constraint.  We also need to set a lower bound constraint on the total abstraction rate (good thing we included all those list file budget components as observations!)

In [ ]:
obs.loc[swgw_hist,"obgnme"] = "less_than"
obs.loc[swgw_hist,"weight"] = 1.0

obs.loc[swgw_hist,"obsval"] = -300

tot_abs_rate = ["flx_wells_19791230"]#,"flx_wells_19801229"]
obs.loc[tot_abs_rate,"obgnme"] = "less_than"
obs.loc[tot_abs_rate,"weight"] = 1.0
obs.loc[tot_abs_rate,"obsval"] = -600.0
pst.less_than_obs_constraints

In [ ]:
pst.control_data.noptmax = 1
pst.write(os.path.join(t_d,"freyberg_opt.pst"))

In [ ]:
pyemu.os_utils.start_slaves(t_d,"pestpp-opt","freyberg_opt.pst",num_slaves=10,master_dir=m_d)

Let's load and inspect the response matrix

In [ ]:
jco = pyemu.Jco.from_binary(os.path.join(m_d,"freyberg_opt.1.jcb")).to_dataframe().loc[pst.less_than_obs_constraints,:]
jco

Let's also load the optimal decision variable values:

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

The sum of these values is the optimal objective function value. However, since these are just mulitpliers on the pumping rate, this number isnt too meaningful. Instead, lets look at the residuals file

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"freyberg_opt.pst"),resfile=os.path.join(m_d,"freyberg_opt.1.rei"))
pst.res.loc[pst.nnz_obs_names,:]

Sweet as!  lots of room in the optimization problem.  The bounding constraint is the one closest to its RHS

### Opt under uncertainty part 1: FOSM chance constraints

This is where the process of uncertainty quantification/history matching and mgmt optimizatiom meet - worlds collide! 

Mechanically, in PESTPP-OPT, to activate the chance constraint process, we need to specific a risk != 0.5

In [ ]:
pst.pestpp_options["opt_risk"] = 0.4

For the FOSM-based chance constraints, we also need to have at least one adjustable non-dec-var parameter so that we can propogate parameter uncertainty to model-based constraints (this can also be posterior FOSM is non-constraint, non-zero-weight observations are specified).  For this simple demo, lets just use the constant multiplier parameters in the prior uncertainty stance:

In [ ]:
cn_pars = par.loc[par.pargp.apply(lambda x: "cn" in x),"parnme"]
cn_pars

In [ ]:
par = pst.parameter_data
par.loc[cn_pars,"partrans"] = "log"
pst.control_data.noptmax = 1
pst.write(os.path.join(t_d,"freyberg_opt_uu1.pst"))
pst.npar_adj

In [ ]:
pyemu.os_utils.start_slaves(t_d,"pestpp-opt","freyberg_opt_uu1.pst",num_slaves=20,master_dir=m_d)

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"freyberg_opt_uu1.pst"),resfile=os.path.join(m_d,"freyberg_opt_uu1.1.rei"))
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_uu1.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

### Opt under uncertainty part 2: ensemble-based chance constraints

PESTPP-OPT can also skip the FOSM calculations if users specify model-based constraint weights as standard deviations.  These can be derived from existing ensembles (oh snap!)

In [ ]:
obs_df = pd.read_csv(os.path.join("master_prior_sweep","sweep_out.csv"),index_col=0)
obs_df = obs_df.loc[obs_df.failed_flag==0,:]

In [ ]:
pr_std = obs_df.std().loc[pst.nnz_obs_names]
pr_std

Wait!  Something is wrong here:  The cumulative well flux constraint is not uncertain - it is just a summation of the specified flux.  So lets give it a crazy small weight, implying it has a tiny uncertainty

In [ ]:
pr_std["flx_wells_19791230"] = 1.0e-10
pr_std

In [ ]:
pst.observation_data.loc[pst.nnz_obs_names,"weight"] = pr_std.loc[pst.nnz_obs_names]
pst.pestpp_options["opt_std_weights"] = True
pst.write(os.path.join(t_d,"freyberg_opt_uu2.pst"))

In [ ]:
pyemu.os_utils.start_slaves(t_d,"pestpp-opt","freyberg_opt_uu2.pst",num_slaves=10,master_dir=m_d)

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_uu2.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

### Super secret mode

turns out, if the opt problem is truely linear, we can reuse results of a previous PESTPP-OPT run to modify lots of the pieces of the optimization problem and resolve without running the model even once!  WAT!?

In [ ]:
shutil.copy2(os.path.join(m_d,"freyberg_opt_uu2.1.jcb"),os.path.join(m_d,"restart.jcb"))
shutil.copy2(os.path.join(m_d,"freyberg_opt_uu2.jcb.1.rei"),os.path.join(m_d,"restart.rei"))

pst.pestpp_options["base_jacobian"] = "restart.jcb"
pst.pestpp_options["hotstart_resfile"] = "restart.rei"
pst.pestpp_options["opt_skip_final"] = True
pst.write(os.path.join(m_d,"freyberg_opt_restart.pst"))

In [ ]:
pyemu.os_utils.run("pestpp-opt freyberg_opt_restart.pst",cwd=m_d)

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_restart.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

Oh snap!  that means we can do all sort of kewl optimization testing really really fast....

In [ ]:
pst.pestpp_options["opt_risk"] = 0.54
pst.write(os.path.join(m_d,"freyberg_opt_restart.pst"))
pyemu.os_utils.run("pestpp-opt freyberg_opt_restart.pst",cwd=m_d)
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_restart.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

Lets use the functionality to evaluate how our OUU problem changes if we use posterior standard deviations:

In [ ]:
obs_df = pd.read_csv(os.path.join("master_ies","freyberg_ies.3.obs.csv"),index_col=0)
pt_std = obs_df.std().loc[pst.nnz_obs_names]
pt_std["flx_wells_19791230"] = 1.0e-10
pt_std

In [ ]:
pst.observation_data.loc[pst.nnz_obs_names,"weight"] = pt_std.loc[pst.nnz_obs_names]
pst.observation_data.loc[pst.nnz_obs_names,"weight"]

In [ ]:
pst.write(os.path.join(m_d,"freyberg_opt_restart.pst"))
pyemu.os_utils.run("pestpp-opt freyberg_opt_restart.pst",cwd=m_d)
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_restart.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

In [ ]:
res_df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_opt_restart.jcb.1.rei")).loc[pst.nnz_obs_names,:]
res_df

In [ ]:
pst.pestpp_options["opt_risk"] = 0.95
pst.write(os.path.join(m_d,"freyberg_opt_restart.pst"))
pyemu.os_utils.run("pestpp-opt freyberg_opt_restart.pst",cwd=m_d)
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_restart.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

# FINALLY!!!

We now see the reason for high-dimensional uncertainty quantification and history matching: to define and the reduce (through data assimulation) the uncertainty in the model-based constraints so that we can find a 95% risk-averse management solution.  BOOM!